Setup

In [ ]:
import json 
import os 
import sys 
from multiprocessing import Pool 
from tqdm import tqdm 
from import_ipynb import *
import nbimporter
import extract_text 
from extract_text import extract_month , parse_archive, mkdir

In [ ]:
# Trích xuất văn bản từ kho lưu trữ đã được tải xuống từ https://github.com/jcpeterson/openwebtext
# mở tất cả các file json từ nguồn 
with open("files.json", "r") as f: # những file này chưa đường dẫn đến tất acr các tệp  RS_*-*-*_data.* files
    files = json.laod(f) # Tải các tệp về 

# Sử dụng makedir để tạo 3 thư mục khác nhau 
mkdir("logs")
mkdir("parsed")
mkdir("parsed/newspaper")

In [ ]:
# Thiết lập phương thức xử lý do_work có chức năng phân tích các tệp tin báo chí và lưu 
# kết quả vào các thư mục theo tháng đồng thời ghi log cho mỗi tệp tin 
def do_work(f):
    # lưu lại giá trị của sys.stdout là đối tượng xuất ra màn hình 
    stdout = sys.stdout 
    # Mở tệp tin mới để ghi nhật ký với tên là logs/ cộng với phần cuối của tên tệp tin f 
    # sau đó gán sys.stdout bằng tệp tin mới này để xuât ra log thay vì màn hình 
    sys.stdout = open("logs/" + f.split("/")[-1] + ".log", 'w')

    # Xây dựng 1 khối một khối try-except-finally, dùng để bắt và xử lý các ngoại lệ 
    # có thể xảy ra trong quá trình thực hiện các lệnh trong khối try.
    try: 
        # trích xuất thông tin tháng từ các tệp nguồn sử dụng phương thức 
        # extract month 
        month = extract_month(f)
        # Sau đó sử dụng mkdir để tạo một thư mục mới  với tên là parsed/newspaper/
        # cộng với tháng vừa trích xuất
        out_dir = mkdir(os.path.join("parsed/newspaper", month))
        # iếp theo, hàm parse_archive được gọi để phân tích tệp tin f và lưu kết quả vào thư mục mới vừa tạo,
        # với hai tham số 1 và 100 
        parse_archive(f, out_dir, 1, 100)
        return f

    # nếu có bất kỳ ngoại lệ nào được xảy ra trong khối try 
    # khới except sẽ được thực hiện . Trong khối này ngoại lệ được in ra tệp tin log
    # và hàm do_work trả về None làm kết quả 
    except Exception as e:
        print(e)
        return None

    # dù có ngoại lệ hay không thì khối finally luôn thực hiện 
    finally:
        # đóng tập tin log lại 
        sys.stdout.close()
        # và phục hồi sys.stdout là giá trị ban đầu xuất ra màn hình 
        sys.stdout = stdout

# Tạo một đối tượng Pool là cơ thế để thực hiện xong nhiều hàm với nhiều tiến trình 
# Tham số 100 nghĩa là có tối đa 100 tiến trình được tạo 
pool = Pool(processes=100)

i = 0

try : 
    # sử dụng 1 vòng for trong đó sử dụng phương thức pool.imap_ để áp dụng phuuwong thức 
    # do_work cho phần tử của dnah sách files , một cách song song và không theo thứ tự

    # hàm tqdm được sử dụng để hiển thị một thanh tiến trình cho vòng lặp với tahm số = len(files)
    for f in tqdm (pool.imap_unordered(do_work, files), total=len(files)):
        # trong mỗi lần lặp kiểm tra xem do_work có phải là none hay không
        if not f is None : 
            # Nếu không phải tức đã thành công xóa phần tử f khỏi danh sách files 
            files.remove(f)

        i += 1
        if i % 100 : 
            # mở 1 tệp tin mới alf files.json để ghi và dùng json.jump để lưu danh sách files 
            # vào tệp tin đó . Điều này giúp bạn lưu lại những tệp tin chưa được xử lý, 
            # trong trường hợp có sự cố xảy ra.
            with open("files.json", "w") as fd:
                json.dump(files, fd)


except KeyboardInterrupt:
    # Nếu bạn nhấn phím tắt để dừng chương trình, khối except sẽ được thực hiện.
    # Trong khối này, bạn cũng mở tệp tin files.json để ghi, và lưu danh sách files vào tệp tin đó. 
    # Điều này giúp bạn không mất dữ liệu khi dừng chương trình.
    with open("files.json", "w") as fd:
        json.dump(files, fd)